# Modelos de aprendizaje automático


In [1]:
import pyspark
import numpy as np
import pandas as pd
import sys
import os

print (pyspark.__version__)
print (np.__version__)
print (pd.__version__)
print (sys.version)

4.0.0
1.26.0
2.3.2
3.11.0rc1 (main, Aug 12 2022, 10:02:14) [GCC 11.2.0]


In [2]:
entornos = []
#local = ('local[*]','AppCLusterLocal','/Users/Juanma/Desktop/work-consolidado/01-2024.csv')
local = ('local[*]','AppCLusterLocal','/media/juanma/spark/datos/01-2024.csv')
compose = ('spark://master:7077','AppCLusterTFM','/mnt/volume/cvs_ok/01-2024.csv')
entornos.append(local)
entornos.append(compose)
fichero = ''
entornos


from pyspark.sql import SparkSession
import pandas as pd

global spark

def ConexionCluster(entorno=0):
# jobs: http://localhost:4040/jobs/
    global fichero    
    srv = entornos[entorno][0]
    app = entornos[entorno][1]
    fichero = entornos[entorno][2]    
    print (srv)
    print (app)
    print (fichero)
    print ('entorno', entorno)
    if entorno > 0:        
        #.config("spark.executor.memory", "16g") \        
        return SparkSession.builder \
        .appName(app) \
        .master(srv) \
        .config("spark.driver.memory", "4g") \
        .config("spark.executor.memory", "6g") \
        .config("spark.executor.cores", 2) \
        .config("spark.cores.max", 10) \
        .config("spark.hadoop.fs.defaultFS", "hdfs://master:9000") \
        .config("spark.driver.maxResultSize", "4g") \
        .getOrCreate()
        #.config("spark.local.dir", "/mnt/volume/spark-tmp") \
    else:
        return SparkSession.builder \
            .appName(app) \
            .master(srv) \
            .config("spark.local.dir", "/mnt/volume/spark-tmp") \
            .getOrCreate()        

#    .config("spark.pyspark.python", "/usr/bin/python3.11") \
#    .config("spark.pyspark.driver.python", "/usr/bin/python3.11") \

def CargaMuestra():
    cad = "file:" + fichero
    print (cad)
    return spark.read.format("csv").option("inferSchema","true").option("sep",";").option("header","true") \
    .load(cad)



'''spark = (
    SparkSession.builder
    .appName("ColabSpark4.0.0")
    # Increase driver memory (e.g., to 2g or more depending on your data size)
    .config("spark.driver.memory", "12g")   # increase driver memory
    #.config("spark.executor.memory", "8g") # increase executor memory
    .config("spark.driver.maxResultSize", "2g")  # avoid huge result collection
    # Increase executor memory if you have executors (not applicable in local mode usually)
    # .config("spark.executor.memory", "2g")
    # Add Delta Lake package configuration
    #.config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0")
    #.config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    #.config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)
'''
''''''


''

## Conectamos al cluster y hacemos algunas pruebas

In [3]:

spark = ConexionCluster(1)

# =========================================
# Pruebas...
# =========================================
df = spark.createDataFrame(
    [(1, "Málaga"), (2, "Madrid"), (3, "Cádiz")],
    ["id", "ciudad"]
)
df.show()


print("Spark Context Master:", spark.sparkContext.master)

# Test simple: contar palabras
rdd = spark.sparkContext.parallelize(["hola mundo", "esto es una prueba", "spark con docker"])
word_counts = rdd.flatMap(lambda line: line.split(" ")) \
                 .map(lambda word: (word, 1)) \
                 .reduceByKey(lambda a, b: a + b)

print("Resultados:")
print(word_counts.collect())

spark.conf.set("spark.sql.files.maxPartitionBytes", 134217728)  
spark.conf.set("spark.sql.shuffle.partitions", 8)               

spark.version

spark://master:7077
AppCLusterTFM
/mnt/volume/cvs_ok/01-2024.csv
entorno 1


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/08 23:29:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

+---+------+
| id|ciudad|
+---+------+
|  1|Málaga|
|  2|Madrid|
|  3| Cádiz|
+---+------+

Spark Context Master: spark://master:7077
Resultados:
[('es', 1), ('docker', 1), ('esto', 1), ('hola', 1), ('spark', 1), ('con', 1), ('una', 1), ('mundo', 1), ('prueba', 1)]


'4.0.0'

In [4]:
spark

## Prueba de MINST con torchdistributor

In [5]:
from pyspark.sql import SparkSession
from pyspark.ml.torch.distributor import TorchDistributor
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, DistributedSampler
import os
import torch.distributed as dist


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)


def main():
    dist.init_process_group(backend="gloo")

    rank = dist.get_rank()
    world_size = dist.get_world_size()

    torch.manual_seed(0)

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    train_dataset = datasets.MNIST('/tmp/mnist', train=True, download=True, transform=transform)
    sampler = DistributedSampler(train_dataset, num_replicas=world_size, rank=rank)
    train_loader = DataLoader(train_dataset, batch_size=64, sampler=sampler)

    model = Net()
    model = nn.parallel.DistributedDataParallel(model)

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(1, 20):  # solo 2 epochs para prueba
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % 100 == 0 and rank == 0:
                print(f"[Epoch {epoch}] Batch {batch_idx} Loss: {loss.item()}")
    dist.destroy_process_group()

distributor = TorchDistributor(
    #num_processes=3,  # spark.sparkContext.defaultParallelism
    num_processes=spark.sparkContext.defaultParallelism,
    local_mode = False,
    use_gpu=False)

distributor.run(main)



Started distributed training with 10 executor processes
[Epoch 1] Batch 0 Loss: 2.3251395225524902                        (0 + 10) / 10]
[Epoch 2] Batch 0 Loss: 0.20318341255187988
[Epoch 3] Batch 0 Loss: 0.15609514713287354
[Epoch 4] Batch 0 Loss: 0.12071896344423294
[Epoch 5] Batch 0 Loss: 0.09943787008523941
[Epoch 6] Batch 0 Loss: 0.08340492099523544
[Epoch 7] Batch 0 Loss: 0.07449784874916077
[Epoch 8] Batch 0 Loss: 0.06500767916440964
[Epoch 9] Batch 0 Loss: 0.05636418238282204
[Epoch 10] Batch 0 Loss: 0.045968152582645416
[Epoch 11] Batch 0 Loss: 0.03986917808651924
[Epoch 12] Batch 0 Loss: 0.035076674073934555
[Epoch 13] Batch 0 Loss: 0.02744406647980213
[Epoch 14] Batch 0 Loss: 0.027873048558831215
[Epoch 15] Batch 0 Loss: 0.028492441400885582
[Epoch 16] Batch 0 Loss: 0.02563989907503128
[Epoch 17] Batch 0 Loss: 0.024664783850312233
[Epoch 18] Batch 0 Loss: 0.01674218475818634
[Epoch 19] Batch 0 Loss: 0.018313443288207054
Finished distributed training with 10 executor processe

In [6]:
# my_df.write.format("parquet").mode("overwrite").save("hdfs://master:9000/medidas/2025Q1")
my_df = spark.read.parquet("hdfs://master:9000/medidas/2025Q1")
#my_df = spark.read.parquet("file:///mnt/volume/medidas_2025Q1")
#my_df.write.format("parquet").mode("overwrite").save("file:///mnt/volume/medidas_2025Q1")
#my_df.show()

In [7]:
#my_df.write.format("parquet").mode("overwrite").save("file:///mnt/volume/medidas_2025Q1_double")

In [8]:
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType, IntegerType

my_df.printSchema()
columns_with_types = my_df.dtypes
integer_cols = [name for name, dtype in columns_with_types if dtype == 'int']

for col_name in integer_cols:
    my_df = my_df.withColumn(col_name, col(col_name).cast(DoubleType()))

my_df.printSchema()

root
 |-- fecha: string (nullable = true)
 |-- tipo_elem: string (nullable = true)
 |-- intensidad: integer (nullable = true)
 |-- ocupacion: double (nullable = true)
 |-- carga: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- min: integer (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- week_of_year: integer (nullable = true)
 |-- num_min: integer (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- longitud: double (nullable = true)
 |-- latitud: double (nullable = true)
 |-- festivo: string (nullable = true)

root
 |-- fecha: string (nullable = true)
 |-- tipo_elem: string (nullable = true)
 |-- intensidad: double (nullable = true)
 |-- ocupacion: double (nullable = true)
 |-- carga: double (nullable = true)
 |-- year: double (nullable = true)
 |-- month: double (nullable = true)
 |-- day: double (nullable = true)
 |-- hour: d

In [9]:
# Grabamos los ficheros de datos:

my_df.cache()
my_df.write.format("parquet").mode("overwrite").save("hdfs://master:9000/my_df_double")
my_df.show()

+----------+---------+----------+---------+-----+------+-----+----+----+----+-----------+------------+-------+-------+-----------------+-----------------+-------+
|     fecha|tipo_elem|intensidad|ocupacion|carga|  year|month| day|hour| min|day_of_week|week_of_year|num_min|quarter|         longitud|          latitud|festivo|
+----------+---------+----------+---------+-----+------+-----+----+----+----+-----------+------------+-------+-------+-----------------+-----------------+-------+
|2025-03-27|      URB|     117.0|      4.0| 29.0|2025.0|  3.0|27.0| 9.0|45.0|        4.0|        13.0|  585.0|    1.0|-3.65341910212066|40.41227295169022|      N|
|2025-03-27|      URB|      75.0|      7.0| 24.0|2025.0|  3.0|27.0|11.0|45.0|        4.0|        13.0|  705.0|    1.0|-3.65341910212066|40.41227295169022|      N|
|2025-03-27|      URB|      50.0|      4.0| 14.0|2025.0|  3.0|27.0|12.0| 0.0|        4.0|        13.0|  720.0|    1.0|-3.65341910212066|40.41227295169022|      N|
|2025-03-27|      URB|

In [10]:
#my_df = spark.read.parquet("hdfs://master:9000/medidas/2025Q1")
#my_df.cache()

In [11]:
my_df.count()

38458217

In [12]:
my_df.printSchema()

root
 |-- fecha: string (nullable = true)
 |-- tipo_elem: string (nullable = true)
 |-- intensidad: double (nullable = true)
 |-- ocupacion: double (nullable = true)
 |-- carga: double (nullable = true)
 |-- year: double (nullable = true)
 |-- month: double (nullable = true)
 |-- day: double (nullable = true)
 |-- hour: double (nullable = true)
 |-- min: double (nullable = true)
 |-- day_of_week: double (nullable = true)
 |-- week_of_year: double (nullable = true)
 |-- num_min: double (nullable = true)
 |-- quarter: double (nullable = true)
 |-- longitud: double (nullable = true)
 |-- latitud: double (nullable = true)
 |-- festivo: string (nullable = true)



# Modelo de aprendizaje automático 

In [13]:
my_df = spark.read.parquet("hdfs://master:9000/medidas/2025Q1")
my_df.cache()
my_df.count()

38458217

In [14]:
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType, IntegerType

my_df.printSchema()
columns_with_types = my_df.dtypes
integer_cols = [name for name, dtype in columns_with_types if dtype == 'int']

for col_name in integer_cols:
    my_df = my_df.withColumn(col_name, col(col_name).cast(DoubleType()))

my_df.printSchema()


my_df.cache()
my_df.write.format("parquet").mode("overwrite").save("hdfs://master:9000/my_df_double")

25/09/08 23:31:11 WARN CacheManager: Asked to cache already cached data.


root
 |-- fecha: string (nullable = true)
 |-- tipo_elem: string (nullable = true)
 |-- intensidad: integer (nullable = true)
 |-- ocupacion: double (nullable = true)
 |-- carga: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- min: integer (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- week_of_year: integer (nullable = true)
 |-- num_min: integer (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- longitud: double (nullable = true)
 |-- latitud: double (nullable = true)
 |-- festivo: string (nullable = true)

root
 |-- fecha: string (nullable = true)
 |-- tipo_elem: string (nullable = true)
 |-- intensidad: double (nullable = true)
 |-- ocupacion: double (nullable = true)
 |-- carga: double (nullable = true)
 |-- year: double (nullable = true)
 |-- month: double (nullable = true)
 |-- day: double (nullable = true)
 |-- hour: d

In [15]:
my_df = my_df.filter('month = 1')
my_df.show()
my_df.count()

+----------+---------+----------+---------+-----+------+-----+---+----+----+-----------+------------+-------+-------+-----------------+-----------------+-------+
|     fecha|tipo_elem|intensidad|ocupacion|carga|  year|month|day|hour| min|day_of_week|week_of_year|num_min|quarter|         longitud|          latitud|festivo|
+----------+---------+----------+---------+-----+------+-----+---+----+----+-----------+------------+-------+-------+-----------------+-----------------+-------+
|2025-01-02|      C30|     516.0|      2.0|  0.0|2025.0|  1.0|2.0| 0.0|15.0|        4.0|         1.0|   15.0|    1.0|-3.74683415991325|40.40682430154291|      N|
|2025-01-02|      C30|     348.0|      2.0|  0.0|2025.0|  1.0|2.0| 0.0|30.0|        4.0|         1.0|   30.0|    1.0|-3.74683415991325|40.40682430154291|      N|
|2025-01-02|      C30|     216.0|      2.0|  0.0|2025.0|  1.0|2.0| 2.0| 0.0|        4.0|         1.0|  120.0|    1.0|-3.74683415991325|40.40682430154291|      N|
|2025-01-02|      C30|     1

13181133

In [16]:
my_df.show()

+----------+---------+----------+---------+-----+------+-----+---+----+----+-----------+------------+-------+-------+-----------------+-----------------+-------+
|     fecha|tipo_elem|intensidad|ocupacion|carga|  year|month|day|hour| min|day_of_week|week_of_year|num_min|quarter|         longitud|          latitud|festivo|
+----------+---------+----------+---------+-----+------+-----+---+----+----+-----------+------------+-------+-------+-----------------+-----------------+-------+
|2025-01-02|      C30|     516.0|      2.0|  0.0|2025.0|  1.0|2.0| 0.0|15.0|        4.0|         1.0|   15.0|    1.0|-3.74683415991325|40.40682430154291|      N|
|2025-01-02|      C30|     348.0|      2.0|  0.0|2025.0|  1.0|2.0| 0.0|30.0|        4.0|         1.0|   30.0|    1.0|-3.74683415991325|40.40682430154291|      N|
|2025-01-02|      C30|     216.0|      2.0|  0.0|2025.0|  1.0|2.0| 2.0| 0.0|        4.0|         1.0|  120.0|    1.0|-3.74683415991325|40.40682430154291|      N|
|2025-01-02|      C30|     1

### Separamos el conjunto de datos en entrenamiento y test

In [17]:
# Separamos en train y test
train_data, test_data = my_df.randomSplit([0.8, 0.2], seed=42) 

print("Entrenamiento: ", train_data.count())
print("Test: ", test_data.count())

Entrenamiento:  10544713


[Stage 28:======================================>                  (8 + 4) / 12]

Test:  2636420


In [18]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

categorical_cols = ['tipo_elem', 'festivo']

# Omitimos carga y ocupación
numerical_cols = [col for col, dtype in my_df.dtypes if dtype in ['int', 'double', 'float', 'long'] and col not in ['intensidad', 'carga','ocupacion']]

print("Categoricas:", categorical_cols)
print("Numericas:", numerical_cols)

Categoricas: ['tipo_elem', 'festivo']
Numericas: ['year', 'month', 'day', 'hour', 'min', 'day_of_week', 'week_of_year', 'num_min', 'quarter', 'longitud', 'latitud']


### Codificamos las columnas categóricas y montamos un vector con los datos de entrada

In [19]:
from pyspark.ml import PipelineModel
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler

indexers = [StringIndexer(inputCol=col, outputCol=col + "_indexed", handleInvalid="keep") for col in categorical_cols]
encoders = [OneHotEncoder(inputCol=col + "_indexed", outputCol=col + "_encoded") for col in categorical_cols]
feature_cols = [col + "_encoded" for col in categorical_cols] + numerical_cols
assembler = VectorAssembler(inputCols=feature_cols, outputCol='raw_features')
scaler = StandardScaler(inputCol="raw_features", outputCol="features", withStd=True, withMean=True)

feature_pipeline = Pipeline(stages=indexers + encoders + [assembler])
feature_model = feature_pipeline.fit(train_data)

train_data_features = feature_model.transform(train_data)
test_data_features = feature_model.transform(test_data)

scaler_model = scaler.fit(train_data_features)

train_data_scaled = scaler_model.transform(train_data_features).select('features', 'intensidad')
test_data_scaled = scaler_model.transform(test_data_features).select('features', 'intensidad')

train_data_scaled.cache()
test_data_scaled.cache()

print("Schema of scaled training data:")
train_data_scaled.printSchema()
train_data_scaled.show(5)

test_data_scaled.printSchema()
test_data_scaled.show(5)

train_data_scaled.createOrReplaceTempView("train_scaled")
test_data_scaled.createOrReplaceTempView("test_scaled")
spark.sql('select count(*) from train_scaled where intensidad = 0').show()
spark.sql('select count(*) from train_scaled where intensidad > 0').show()
spark.sql('select count(*) from test_scaled where intensidad = 0').show()
spark.sql('select count(*) from test_scaled where intensidad > 0').show()

train_data_scaled.write.format("parquet").mode("overwrite").save("hdfs://master:9000/train_data_scaled")
test_data_scaled.write.format("parquet").mode("overwrite").save("hdfs://master:9000/test_data_scaled")

25/09/08 23:31:39 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

Schema of scaled training data:
root
 |-- features: vector (nullable = true)
 |-- intensidad: double (nullable = true)



+--------------------+----------+
|            features|intensidad|
+--------------------+----------+
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
+--------------------+----------+
only showing top 5 rows
root
 |-- features: vector (nullable = true)
 |-- intensidad: double (nullable = true)



+--------------------+----------+
|            features|intensidad|
+--------------------+----------+
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
+--------------------+----------+
only showing top 5 rows


+--------+
|count(1)|
+--------+
|  222942|
+--------+

+--------+
|count(1)|
+--------+
|10321771|
+--------+



+--------+
|count(1)|
+--------+
|   55631|
+--------+

+--------+
|count(1)|
+--------+
| 2580789|
+--------+



In [20]:
print (numerical_cols)
print (categorical_cols)

['year', 'month', 'day', 'hour', 'min', 'day_of_week', 'week_of_year', 'num_min', 'quarter', 'longitud', 'latitud']
['tipo_elem', 'festivo']


In [23]:
my_df.cache()
my_df.createOrReplaceTempView("tb_my_df")
spark.sql('select count(*) from tb_my_df where intensidad = 0').show()
spark.sql('select count(*) from tb_my_df where intensidad > 0').show()

25/09/08 23:32:41 WARN CacheManager: Asked to cache already cached data.


+--------+
|count(1)|
+--------+
|  278573|
+--------+

+--------+
|count(1)|
+--------+
|12902560|
+--------+



In [24]:
train_data = train_data_scaled
test_data = test_data_scaled
train_data.cache()
test_data.cache()
train_data.show()
test_data.show()

+--------------------+----------+
|            features|intensidad|
+--------------------+----------+
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...|       0.0|
+--------------------+----------+
only showing top 20 rows
+--------------------+----------+
|            features|intensidad|
+--------------------+----------+
|[-3.2008233181527...|       0.0|
|[-3.2008233181527...| 

25/09/08 23:32:42 WARN CacheManager: Asked to cache already cached data.
25/09/08 23:32:42 WARN CacheManager: Asked to cache already cached data.


## Modelo básico de regresión lineal.

In [25]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler

lr = LinearRegression(featuresCol='features', labelCol='intensidad')
pipeline = Pipeline(stages=[lr])


In [31]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline

#lr = LinearRegression(featuresCol='features', labelCol='intensidad')

lr = LinearRegression(featuresCol="features", labelCol="intensidad", regParam=0.1)

pipeline = Pipeline(stages=[lr])

In [32]:
model = pipeline.fit(train_data)


In [33]:
from pyspark.ml.evaluation import RegressionEvaluator

pred = model.transform(test_data)
evaluator = RegressionEvaluator(labelCol='intensidad', predictionCol='prediction')
rmse = evaluator.evaluate(pred, {evaluator.metricName: "rmse"})
print(f"(RMSE) test data: {rmse}")
r2 = evaluator.evaluate(pred, {evaluator.metricName: "r2"})
print(f"R-squared test data: {r2}")


(RMSE) test data: 493.22533042258334
R-squared test data: 0.25139544300076


In [34]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml import Pipeline

gbt = GBTRegressor(
    featuresCol='features',
    labelCol='intensidad',
    maxIter=100,       
    maxDepth=5,        
    stepSize=0.1,    
    subsamplingRate=0.8, 
    minInstancesPerNode=10, 
    lossType="squared"
)
gbt_pipeline = Pipeline(stages=[gbt])


In [35]:
model_gbt = gbt_pipeline.fit(train_data_scaled)

In [62]:
from pyspark.ml.evaluation import RegressionEvaluator
pred = model_gbt.transform(test_data)
evaluator_gbt = RegressionEvaluator(labelCol='intensidad', predictionCol='prediction')
rmse = evaluator_gbt.evaluate(pred, {evaluator_gbt.metricName: "rmse"})
print(f"RMSE (gbt): {rmse}")
r2 = evaluator_gbt.evaluate(pred, {evaluator_gbt.metricName: "r2"})
print(f"R-squared (gbt): {r2}")

RMSE (gbt): 384.00127502775626


[Stage 2436:=========================================>             (9 + 3) / 12]

R-squared (gbt): 0.5464904872985833


In [63]:
train_data.cache()

25/09/07 21:07:59 WARN CacheManager: Asked to cache already cached data.


DataFrame[features: vector, intensidad: double]

In [64]:
train_data.show()

+--------------------+----------+
|            features|intensidad|
+--------------------+----------+
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
+--------------------+----------+
only showing top 20 rows


In [65]:
copia_train_data = train_data
copia_test_data = test_data
train_data.show()
test_data.show()

+--------------------+----------+
|            features|intensidad|
+--------------------+----------+
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...|       0.0|
+--------------------+----------+
only showing top 20 rows
+--------------------+----------+
|            features|intensidad|
+--------------------+----------+
|[-3.2008766170360...|       0.0|
|[-3.2008766170360...| 

### Preparamos modelo pytorch regresión simple

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.functions import vector_to_array
from pyspark.ml.torch.distributor import TorchDistributor


train_out = (train_data
             .withColumn("features_arr", vector_to_array(col("features")))
             .select("features_arr", "intensidad"))
test_out = (test_data
            .withColumn("features_arr", vector_to_array(col("features")))
            .select("features_arr", "intensidad"))

train_path = "file:///mnt/volume/tempo/train_parquet"
test_path  = "file:///mnt/volume/tempo/test_parquet"

train_out.write.mode("overwrite").parquet(train_path)
test_out.write.mode("overwrite").parquet(test_path)


def train_main_fn():
    import os, torch, torch.nn as nn, torch.optim as optim
    import torch.distributed as dist
    from torch.utils.data import DataLoader, TensorDataset
    from torch.utils.data.distributed import DistributedSampler
    import pyarrow.parquet as pq
    import numpy as np


    backend = "gloo"   # use "nccl" if GPUs are available
    dist.init_process_group(backend=backend)
    rank = int(os.environ["RANK"])
    world_size = int(os.environ["WORLD_SIZE"])

    TRAIN_PATH = "file:///mnt/volume/tempo/train_parquet"
    TEST_PATH  = "file:///mnt/volume/tempo/test_parquet"
    
    train_tbl = pq.read_table(TRAIN_PATH)
    test_tbl  = pq.read_table(TEST_PATH)

    X_train = np.stack(train_tbl["features_arr"].to_pylist()).astype("float32")
    y_train = np.asarray(train_tbl["intensidad"].to_pylist(), dtype="float32")
    X_test  = np.stack(test_tbl["features_arr"].to_pylist()).astype("float32")
    y_test  = np.asarray(test_tbl["intensidad"].to_pylist(), dtype="float32")

    train_ds = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
    test_ds  = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))

    train_sampler = DistributedSampler(train_ds, num_replicas=world_size, rank=rank, shuffle=True)
    test_sampler  = DistributedSampler(test_ds,  num_replicas=world_size, rank=rank, shuffle=False)

    train_loader = DataLoader(train_ds, batch_size=64, sampler=train_sampler, num_workers=0)
    test_loader  = DataLoader(test_ds,  batch_size=64, sampler=test_sampler,  num_workers=0)

    class RegressionModel(nn.Module):
        def __init__(self, in_dim):
            super().__init__()
            self.net = nn.Sequential(
                nn.Linear(in_dim, 128),
                nn.ReLU(),
                nn.Linear(64, 1)
            )
        def forward(self, x): return self.net(x).squeeze(-1)

    model = RegressionModel(in_dim=X_train.shape[1])
    model = nn.parallel.DistributedDataParallel(model)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(25):
        train_sampler.set_epoch(epoch)
        model.train()
        total = 0.0
        for xb, yb in train_loader:
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()
            total += loss.item()
        avg_train_loss = total / len(train_loader)
        print(f"[rank {rank}] epoch {epoch} train_loss={avg_train_loss:.4f}", flush=True)

        model.eval()
        eval_total = 0.0
        with torch.no_grad():
            for xb, yb in test_loader:
                pred = model(xb)
                eval_total += criterion(pred, yb).item()
        avg_test_loss = eval_total / len(test_loader)
        print(f"[rank {rank}] epoch {epoch} test_loss={avg_test_loss:.4f}", flush=True)

    if rank == 0:
        os.makedirs("/mnt/volume/torch_models", exist_ok=True)
        torch.save(model.module.state_dict(), "/mnt/volume/torch_models/regression.pt")
        print("Grabado modelo", flush=True)

    dist.destroy_process_group()

TorchDistributor(
    num_processes=3,
    local_mode=False,
    use_gpu=False
).run(train_main_fn)


In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.torch.distributor import TorchDistributor
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim


train_data.printSchema()
train_data.show(5)

try:
    features_list = train_data.select("features").rdd.map(lambda row: row.features.toArray()).collect()
    features_np = np.array(features_list, dtype=np.float32)
    labels_list = train_data.select("intensidad").rdd.map(lambda row: row.intensidad).collect()
    labels_np = np.array(labels_list, dtype=np.float32).reshape(-1, 1)
except Exception as e:
    print(f"Errpr de datos: {e}")
    spark.stop()
    raise


class RegressionModel(nn.Module):
    def __init__(self, input_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def train_function(features_np, labels_np):
    features_tensor = torch.from_numpy(features_np)
    labels_tensor = torch.from_numpy(labels_np)
    dataset = torch.utils.data.TensorDataset(features_tensor, labels_tensor)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True)

    input_size = features_np.shape[1]
    model = RegressionModel(input_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch_features, batch_labels in dataloader:
            optimizer.zero_grad()
            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(dataloader)}")

    return model

try:
    distributor = TorchDistributor(num_processes=2, local_mode=False, use_gpu=False)
    trained_model = distributor.run(train_function, features_np, labels_np)
    print("Completado. Modelo: ", trained_model)
except Exception as e:
    print(f"Excepción en el entremaniento: {e}")
    spark.stop()
    raise

